In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
X = [[0., 0.], [1., 1.]]

In [4]:
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)  

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [5]:
clf.predict([[-5, 1], [-13., -2.]])

array([0, 0])

In [6]:
[coef.shape for coef in clf.coefs_]

[(2, 5), (5, 2), (2, 1)]

In [2]:
dfStockData = pd.read_json("data/TSLA_stocks.json").transpose()
dfStockData.head()

,1. open,2. high,3. low,4. close,5. volume
2018-11-05,340.50,343.9550,330.140,341.40,7829954.0
2018-11-02,343.74,349.2000,340.910,346.41,7807971.0
2018-11-01,338.26,347.8400,334.725,344.28,8000132.0
2018-10-31,332.54,342.0000,329.100,337.32,7624348.0
2018-10-30,328.39,337.8999,322.260,329.90,9126704.0


In [49]:
for i in range(11275):
    df = json.load(open("data/messages/resp_%s.json" % i))
    dftemp = json_normalize(df['messages'])
    dfTweet = dfTweet.append(dftemp[['body', 'created_at']], ignore_index=True)
print(dfTweet)

                                                     body  \
0       $TSLA very small put add today. Waiting for th...   
1                   $TSLA target 370 for the short Again?   
2               Whole new company now shorts beware $TSLA   
3              $TSLA Scary scary 🐻 TSLA’s coming for you!   
4       $TSLA max pain is 315 for expiry 2018-11-02 So...   
5       $TSLA Happy Halloween all. And this one is ded...   
6       $TSLA the action this week builds confidence. ...   
7       $TSLA AUTO PARK!! yes!!! When I gotta grab a q...   
8       $TSLA Charts show 344 is where this maxes out ...   
9                         $TSLA I love your kinda crazy 😜   
10      $TSLA jumps to #1 in global plug-in electric c...   
11      10-31-2018 Potential Whale Trades for $TSLA (L...   
12                       $TSLA chairmen announcement soon   
13      $TSLA there is no truth to the rumor I share t...   
14      $TSLA everyone want&#39;s the treat but will i...   
15      $TSLA will need 

In [88]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit_transform(dfTweet['body']).todense()
vectorizer.vocabulary_
print(max(vectorizer.vocabulary_))

years


In [58]:
dfTweet.to_csv("clean_tweet_data.csv")

In [59]:
df = pd.read_csv("clean_tweet_data.csv")
df.tail()

,Unnamed: 0,body,created_at
339464,339464,Here’s what 37 Estimize analysts believe $TSLA...,2018-03-11T23:28:45Z
339465,339465,$TSLA $SPY looking at him gets me high! Dude i...,2018-03-11T23:17:02Z
339466,339466,"$TSLA Insiders adding shares recntly, anyways ...",2018-03-11T23:11:54Z
339467,339467,"$TSLA not looking great. $321, $317 take it to...",2018-03-11T22:57:19Z
339468,339468,$TSLA been in this range for almost a year. RS...,2018-03-11T22:54:18Z


In [60]:
df.tail()

,Unnamed: 0,body,created_at
339464,339464,Here’s what 37 Estimize analysts believe $TSLA...,2018-03-11T23:28:45Z
339465,339465,$TSLA $SPY looking at him gets me high! Dude i...,2018-03-11T23:17:02Z
339466,339466,"$TSLA Insiders adding shares recntly, anyways ...",2018-03-11T23:11:54Z
339467,339467,"$TSLA not looking great. $321, $317 take it to...",2018-03-11T22:57:19Z
339468,339468,$TSLA been in this range for almost a year. RS...,2018-03-11T22:54:18Z


-----

In [5]:
dfStockData["diff"] = dfStockData["4. close"] - dfStockData["1. open"]

In [10]:
dfStockMin = dfStockData.drop(columns=["1. open", "2. high", "3. low", "4. close", "5. volume"])

In [20]:
dfStockMin.index.name = "date"
dfStockMin.reset_index(inplace=True)

In [34]:
dfStockMin["date"] = pd.to_datetime(dfStockMin["date"])

In [31]:
# Do min/max normalization on diff values, to get values from -1 to 1
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
vals = dfStockMin["diff"].values
vals = scaler.fit_transform(vals.reshape(-1,1))
dfStockMin["diff"] = vals

In [35]:
dfStockMin.to_csv("normed_stockdata.csv")